In [1]:
from tensorflow import keras
import numpy as np

RND_SEED = 12345

def train_generator():
    return keras.preprocessing.image.ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255.,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
#        rotation_range=90,
    )

def load_train(path):
    return train_generator().flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=RND_SEED)

def create_model(input_shape, weights=None):
    backbone = keras.applications.resnet.ResNet50(
        input_shape=input_shape,
        classes=12,
        include_top=False,
        weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5' if not weights else weights
    )
    backbone.trainable = True
    model = keras.models.Sequential()
    model.add(backbone)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(12, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=15,
               steps_per_epoch=None, validation_steps=None):
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

2025-09-19 15:05:17.283784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-19 15:05:17.336867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-19 15:05:18.563419: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
image_gen = train_generator()
train_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='training',
			seed=RND_SEED)
val_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='validation',
			seed=RND_SEED)

Found 1266 images belonging to 12 classes.
Found 417 images belonging to 12 classes.


In [3]:
features, target = next(train_datagen_flow)
len(train_datagen_flow)

80

In [4]:
print('input shape:', features[0].shape)
model = create_model(features[0].shape, './datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
model = train_model(model,
	train_data=train_datagen_flow,
	test_data=val_datagen_flow)

input shape: (150, 150, 3)


2025-09-19 15:05:19.302423: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/15


/home/wtom/code/notebooks/practicum/vision/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 - 190s - 2s/step - acc: 0.8017 - loss: 0.6769 - val_acc: 0.1295 - val_loss: 3.6460
Epoch 2/15
80/80 - 164s - 2s/step - acc: 0.9787 - loss: 0.0832 - val_acc: 0.1199 - val_loss: 5.6940
Epoch 3/15
80/80 - 164s - 2s/step - acc: 0.9739 - loss: 0.0838 - val_acc: 0.1199 - val_loss: 9.5076
Epoch 4/15
80/80 - 164s - 2s/step - acc: 0.9708 - loss: 0.0959 - val_acc: 0.0743 - val_loss: 5.3238
Epoch 5/15
80/80 - 165s - 2s/step - acc: 0.9858 - loss: 0.0583 - val_acc: 0.0695 - val_loss: 11.8919
Epoch 6/15
80/80 - 165s - 2s/step - acc: 0.9834 - loss: 0.0696 - val_acc: 0.1247 - val_loss: 3.8917
Epoch 7/15
80/80 - 165s - 2s/step - acc: 0.9779 - loss: 0.1025 - val_acc: 0.1031 - val_loss: 6.1381
Epoch 8/15
80/80 - 164s - 2s/step - acc: 0.9866 - loss: 0.0392 - val_acc: 0.2158 - val_loss: 2.9526
Epoch 9/15
80/80 - 166s - 2s/step - acc: 0.9803 - loss: 0.0682 - val_acc: 0.1631 - val_loss: 6.7161
Epoch 10/15
80/80 - 164s - 2s/step - acc: 0.9921 - loss: 0.0250 - val_acc: 0.4125 - val_loss: 2.6158
Epoch 11/